In [15]:
import numpy as np
from skyfield.api import load,Topos
from skyfield.earthlib import terra, reverse_terra

np.random.seed(0)

speed_of_light_km_ns = 0.000299792
au_to_km = 149598000
geo_orbit_km= 35786 


from generator.generate import *

In [16]:
Gll, Vg = GroundStation()
print(Gll)
Vg  = get_cart_xyz(Gll[0], Gll[1]) 
print(Vg, np.linalg.norm(Vg))

[7.8101606283719605, 73.16438456662263]
[1828.09581411 6041.37076166  865.76325552] 6371.0


In [17]:
Tll, Vt = Transmitter(Gll, Vg)
print(Tll)
print(Vt, np.linalg.norm(Vt))
dist = haversine_np(Gll[1], Gll[0], Tll[1], Tll[0])
print(dist)


(7.7702824040134715, 73.20873630462687)
[1823.59237349 6043.35947225  861.36992318] 6370.999999999999
6.598281608115566


In [18]:
Slls, Vs = Satellites(Gll, Vg)
for Sll in Slls:
    print(Sll)
print("")
for V in Vs:
    print(V)

(4.499924066294895, 90.70257617754673)
(17.150172911169047, 82.15442929383902)
(-1.4595699550325136, 82.53415681950788)
(15.06283076520565, 90.56660211825168)
(-0.9328352155642108, 87.86131538344816)

[ -515.4195662  42030.86669785  3308.09360486]
[ 5499.61589699 39912.06108087 12433.20156929]
[ 5476.80686937 41792.99037621 -1073.98227618]
[ -402.63004084 40713.31415109 10957.50135226]
[ 1573.28535346 42129.04537554  -686.44369176]


In [19]:
Rg, Rs = make_rs(Vg, Vs, Vt)
print(Rg)
for R in Rs:
    print(R)


6.598281313221413
36146.34296773767
35979.29106090306
35987.88232250116
36178.63587125097
36119.732940628666


In [22]:
Tg = int(np.ceil(Rg/speed_of_light_km_ns))
Ts = list(map(lambda R: int(np.ceil(R/speed_of_light_km_ns)), Rs))
print(Tg)
for T in Ts:
    print(T)

22010
120571407
120014181
120042838
120679124
120482645


In [23]:
def roundDown(x, size=500):
    return int(np.floor(x/(1.0*size))*size)

Ts = list(map(lambda T : roundDown(T - Tg, size=200), Ts))
Tg = 0
for T in Ts:
    print(T)

120549200
119992000
120020800
120657000
120460600


In [24]:
from solver.solve import *
Rg = Tg * speed_of_light_km_ns
Rs = list(map(lambda T: T * speed_of_light_km_ns, Ts))
print(Rg)
for R in Rs:
    print(R)

0.0
36139.6857664
35972.641664
35981.2756736
36172.003344000004
36113.1241952


In [25]:
guess,nextD = findD(-100,100,0.1,Vg,Vs,Rg,Rs)
print(guess,nextD)
guess,nextD = findD(nextD - 0.1, nextD + 0.1, 0.0001, Vg,Vs,Rg,Rs)
print(guess,nextD)
guess,nextD = findD(nextD - 0.0001, nextD + 0.0001, 0.000001, Vg,Vs,Rg,Rs)
print(guess,nextD)
guess,nextD = findD(nextD - 0.000001, nextD + 0.000001, 0.000000001, Vg,Vs,Rg,Rs)
print(guess,nextD)
guess,nextD = findD(nextD - 0.000000001, nextD + 0.000000001, 0.000000000001, Vg,Vs,Rg,Rs)
print(guess,nextD)

[1823.51832291 6043.38564815  861.45234932] 6.599999999998589
[1823.51697992 6043.37078924  861.45023362] 6.614799999998322
[1823.51697865 6043.37077519  861.45023162] 6.614813999998338
[1823.51697865 6043.37077524  861.45023163] 6.614813949998417
[1823.51697865 6043.37077524  861.45023163] 6.6148139503245345


In [26]:
lat,lon = get_lat_lon(guess)
print(lat,lon)
print(Tll[0], Tll[1])
print(1000*haversine_np(lon,lat, Tll[1], Tll[0]))

7.771011328785729 73.20942109156071
7.7702824040134715 73.20873630462687
110.7320488222988


In [11]:
lat = 30
lon = 50
v = get_cart_xyz(lat,lon)
print(get_lat_lon(v))

(29.999999999999996, 50.0)
